# OQMD Mini Example

In [6]:
from foundry import Foundry

# Instantiate Foundry from local file and load data from remote

In [7]:
f = Foundry()

In [8]:
f = f.from_file()
X,y = f.load_data()

In [9]:
X.head()

,configuration,magnetic_moment,stability,delta_e,total_energy,volume_pa,composition,dft_converged,dft_cutoff_energy,dft_exchange_correlation_functional,crystal_structure_number_of_atoms,crystal_structure_space_group,crystal_structure_volume
0,fine_relax,NaN,NaN,NaN,-6.721376,14.4033,Be2Ir1Pa1,True,252.3,PBE,4,NaN,57.6133
1,standard,NaN,0.060465,-0.618895,-6.536939,20.4781,Ni1Pd1Y2,True,520.0,PBE,4,NaN,81.9125
2,fine_relax,0.412761,NaN,NaN,-5.988129,26.3910,Er2Ta1Tb1,True,223.7,PBE,4,NaN,105.5640
3,standard,1.603898,0.936115,0.936115,-5.704766,24.5555,K1Mn2Th1,True,520.0,PBE,4,NaN,98.2219
4,fine_relax,0.457712,NaN,NaN,-6.387080,25.4143,Pr2Si1Ta1,True,245.3,PBE,4,NaN,101.6570


In [10]:
f.describe()

DC:{}
Dataset:{"inputs": ["configuration", "magnetic_moment", "stability", "delta_e", "total_energy", "volume_pa", "composition", "dft_converged", "dft_cutoff_energy", "dft_exchange_correlation_functional", "crystal_structure_number_of_atoms", "crystal_structure_space_group", "crystal_structure_volume"], "outputs": ["bandgap"], "input_descriptions": [], "output_descriptions": [], "type": "tabular", "hash": [], "version": "0.1", "references": [], "sources": ["https://data.materialsdatafacility.org/public/foundry_dataframe.json"]}


# See Foundry clients

In [11]:
f.dlhub_client

In [12]:
f.forge_client

In [13]:
f.transfer_client

# Run to Create Dataframe and Metadata Locally without Fetch

In [ ]:
mdf = Forge()
r = mdf.match_source_names("oqmd").match_resource_types("record").search(limit=200)

In [ ]:
def oqmd_to_df(res):
    frame = []
    
    for r in res:
        record = {}
        
        # OQMD Block
        record['bandgap'] = r['oqmd'].get('band_gap', {}).get('value', None)
        record['configuration'] = r['oqmd'].get('configuration', None)
        record['magnetic_moment'] = r['oqmd'].get('magnetic_moment',{}).get('value', None)
        record['stability'] = r['oqmd'].get('stability', {}).get('value', None)
        record['delta_e'] = r['oqmd'].get('delta_e', {}).get('value', None)
        record['total_energy'] = r['oqmd'].get('total_energy', {}).get('value', None)
        record['volume_pa'] = r['oqmd'].get('volume_pa', {}).get('value', None)
        
        # Material Block
        record['composition'] = r.get('material', {}).get('composition',"")
        
        #DFT Block
        prefix = "dft_"
        record[prefix+'converged'] = r['dft'].get('converged', None)
        record[prefix+'cutoff_energy'] = r['dft'].get('cutoff_energy', None)
        record[prefix+'exchange_correlation_functional'] = r['dft'].get('exchange_correlation_functional', None)
        
        # Crystal Structure Block
        prefix = "crystal_structure_"
        record[prefix+'number_of_atoms'] = r['crystal_structure'].get('number_of_atoms', None)
        record[prefix+'space_group'] = r['crystal_structure'].get('space_group', None)
        record[prefix+'volume'] = r['crystal_structure'].get('volume', None)

        frame.append(record)
    return frame

In [ ]:
df = pd.DataFrame(oqmd_to_df(r))
df

In [ ]:
oqmd_dict = {
    "dc":{},
    "mdf":{},
    "foundry":{
        "inputs":['configuration',
             'magnetic_moment',
             'stability',
             'delta_e',
             'total_energy',
             'volume_pa',
             'composition',
             'dft_converged',
             'dft_cutoff_energy',
             'dft_exchange_correlation_functional',
             'crystal_structure_number_of_atoms',
             'crystal_structure_space_group',
             'crystal_structure_volume'],
        "outputs":"bandgap",
        "type":"tabular"
    }
}